## Steps of code
1. Extracting data from tvdatafeed
2. Using a statistical approach to generate the labels(1->buy , 0->nothing , -1->sell)
3. dividing the data into train and test data
4. Training the data
5. Comparing the models
6. choosing best model
7. merge the long and short trades in one file
8. Backtesting the result based on the merged file

#### Assumptions
1. During finding the returns we assume to start the trade on open and sqare off of the trade at the close of trade
2. Due to find the solution for the both long and short trades, we are avoiding the stop loss approach till now , buy in future definetly work upon it

## Data Extraction from TradingView using python module tvDatafeed

In [1]:
from tvDatafeed import TvDatafeed, Interval


ModuleNotFoundError: No module named 'tvDatafeed'

In [2]:
with open('Credendials.txt','r') as file:
    content = file.read().split()

FileNotFoundError: [Errno 2] No such file or directory: 'Credendials.txt'

In [ ]:
username = content[0]
password = content[1]

In [ ]:
tv = TvDatafeed(username, password)

In [ ]:
data = tv.get_hist(symbol='BANKNIFTY',exchange='NSE',interval=Interval.in_5_minute,n_bars=10000)

In [ ]:
data.head()

In [ ]:
data.to_csv('BANKNIFTY_till_14thMarch_5min.csv')

### Dividing the data into the training and testing data
we won't be able to use 80% or 20% ratio for training and testing data , because it is time series data the data is more related to back data , that is why we won't be able to follow any specific ratio
So, Solution for this is the way that result in the dividing the dataset depending that training data is around 80% of the total data and also the dependency is minimal

In [4]:
import pandas as pd
final_df = pd.read_csv('Final_data_5min.csv')

In [5]:
len(final_df)

8108

In [6]:
# the value in the below cell 6509 is given to the approx value of 80%, not exactly 80% due to the fact that 

In [7]:
train_data = final_df.iloc[:6509]

In [8]:
test_data = final_df.iloc[6509:]

In [9]:
len(test_data)

1599

In [10]:
train_data.tail()

,datetime,symbol,open,high,low,close,volume
6504,2024-02-13 15:05:00,NSE:BANKNIFTY,45504.90,45529.50,45489.25,45521.00,4875497.0
6505,2024-02-13 15:10:00,NSE:BANKNIFTY,45524.60,45565.95,45501.55,45521.45,5860964.0
6506,2024-02-13 15:15:00,NSE:BANKNIFTY,45521.95,45564.50,45500.75,45500.75,6589835.0
6507,2024-02-13 15:20:00,NSE:BANKNIFTY,45506.00,45517.00,45437.25,45453.00,6617515.0
6508,2024-02-13 15:25:00,NSE:BANKNIFTY,45454.50,45454.50,45383.15,45419.85,4611935.0


In [11]:
test_data.head()

,datetime,symbol,open,high,low,close,volume
6509,2024-02-14 09:15:00,NSE:BANKNIFTY,45095.45,45140.10,44988.70,45034.30,10394636.0
6510,2024-02-14 09:20:00,NSE:BANKNIFTY,45030.25,45119.75,44965.75,44987.65,4113601.0
6511,2024-02-14 09:25:00,NSE:BANKNIFTY,44988.00,44988.00,44880.75,44940.10,6873600.0
6512,2024-02-14 09:30:00,NSE:BANKNIFTY,44942.65,44951.65,44860.75,44913.85,5835451.0
6513,2024-02-14 09:35:00,NSE:BANKNIFTY,44912.70,45070.50,44902.85,45059.60,5188373.0


#### Hence, there is no overlapping in the data between them

In [12]:
train_data.to_csv('Data/Train/train_data.csv')

OSError: Cannot save file into a non-existent directory: 'Data\Train'

In [ ]:
test_data.to_csv('Data/Test/test_data.csv')

### Generating buy signals in the dataset 

In [ ]:
import pandas as pd
import talib as ta
import matplotlib.pyplot as plt
import warnings
import numpy as np

In [ ]:
warnings.filterwarnings('ignore')

In [ ]:
df_train = pd.read_csv('Data/Train/train_data.csv')

In [ ]:
df_test = pd.read_csv('Data/Test/test_data.csv')

In [ ]:
def Indicators(df):
    df['rsi'] = ta.RSI(df['close'],9)
    df['ema3'] = ta.EMA(df['rsi'],6)
    df['wma'] = ta.WMA(df['rsi'],21)
    df['atr'] = ta.ATR(df['high'] , df['low'] , df['close'],7)
    return df

In [ ]:
df_train = Indicators(df_train)
df_test = Indicators(df_test)

In [ ]:
df_train.dropna(inplace=True)
df_test.dropna(inplace=True)

In [ ]:
df_train.head()

In [ ]:
df_train.drop('Unnamed: 0',axis=1,inplace=True)

In [ ]:
df_test.drop('Unnamed: 0',axis=1,inplace=True)

In [ ]:
df_train.head()

In [ ]:
df_test.head()

In [ ]:
len(df_train),len(df_test)

In [ ]:
df_train['rsi'].iloc[0]

In [ ]:
def Generate_buy_signal(df):
    df['buy_signal'] = 0
    for i in range(1,len(df)):
        if df['rsi'].iloc[i] >= df['ema3'].iloc[i] and df['ema3'].iloc[i] >= df['wma'].iloc[i]:
            df['buy_signal'].iloc[i] = 1
    return df

In [ ]:
df_train = Generate_buy_signal(df_train)

In [ ]:
df_train.head(10)

In [ ]:
df_test = Generate_buy_signal(df_test)

In [ ]:
df_test.head(10)

In [ ]:
len(df_train) , len(df_test)

In [ ]:
df_train.to_csv('Buy_Train_data_with_label.csv')

In [ ]:
df_test.to_csv('Buy_Test_data_with_label.csv')

## Generating short sell signals for dataset

In [ ]:
df_train = pd.read_csv('Data/Train/train_data.csv')

In [ ]:
df_test = pd.read_csv('Data/Test/test_data.csv')

In [ ]:
def Indicators(df):
    df['rsi'] = ta.RSI(df['close'],9)
    df['ema3'] = ta.EMA(df['rsi'],6)
    df['wma'] = ta.WMA(df['rsi'],21)
    df['atr'] = ta.ATR(df['high'] , df['low'] , df['close'],7)
    return df

In [ ]:
df_train = Indicators(df_train)

In [ ]:
df_test = Indicators(df_test)

In [ ]:
df_train.dropna(inplace=True)
df_test.dropna(inplace=True)

In [ ]:
df_train.head()

In [ ]:
df_train.drop('Unnamed: 0',axis=1,inplace=True)

In [ ]:
df_test.drop('Unnamed: 0',axis=1,inplace=True)

In [ ]:
def Generate_Short_Sell_signals(df):
    df['Sell_signal'] = 0
    for i in range(1,len(df)):
        if df['wma'].iloc[i] >= df['ema3'].iloc[i] and df['ema3'].iloc[i] >= df['rsi'].iloc[i]:
            df['Sell_signal'].iloc[i] = -1
    return df

In [ ]:
df_train = Generate_Short_Sell_signals(df_train)

In [ ]:
df_test = Generate_Short_Sell_signals(df_test)

In [ ]:
df_train.to_csv('Sell_Train_data_with_label.csv')

In [ ]:
df_test.to_csv('Sell_Test_data_with_label.csv')

In [ ]:
df_test.head(50)

## Now testing the models

In [ ]:
df1 = pd.read_csv('Buy_Train_data_with_label.csv')

In [ ]:
df1.head()

In [ ]:
df2 = pd.read_csv('Sell_Train_data_with_label.csv')

In [ ]:
df3 = pd.read_csv('Buy_Test_data_with_label.csv')

In [ ]:
df4 = pd.read_csv('Sell_Test_data_with_label.csv')

In [ ]:
x1=df1.iloc[:,3:12]
y1=df1.iloc[:,12]

In [ ]:
x1.head()

In [ ]:
y1

In [ ]:
x2 = df2.iloc[:,3:12]
y2 = df2.iloc[:,12]

In [ ]:
x3 = df3.iloc[:,3:12]
y3 = df3.iloc[:,12]

In [ ]:
x4 = df4.iloc[:,3:12]
y4 = df4.iloc[:,12]

In [ ]:
data={}

## Tried Different models to sell where the dataset fits

## SVC

In [ ]:
from sklearn.svm import SVC #SVC- support vector classifier
svc_b=SVC()
svc_s=SVC()

In [ ]:
len(x1) , len(y1)

In [ ]:
y1.head()

In [ ]:
svc_b.fit(x1,y1)

In [ ]:
svc_b.score(x1,y1)

In [ ]:
svc_s.fit(x2,y2)

In [ ]:
data['svc']=[svc_b.score(x3,y3),svc_s.score(x4,y4)]

## Logistic Regression

In [ ]:
from sklearn.linear_model import LogisticRegression
lr_b=LogisticRegression()
lr_s=LogisticRegression()

In [ ]:
lr_b.fit(x1,y1)

In [ ]:
lr_s.fit(x2,y2)

In [ ]:
data['lr']=[lr_b.score(x3,y3),lr_s.score(x4,y4)]

In [ ]:
lr_b.score(x3,y3)

## Decision Tree Classifier

In [ ]:
from sklearn.tree import DecisionTreeClassifier
dtc_b=DecisionTreeClassifier()
dtc_s=DecisionTreeClassifier()

In [ ]:
dtc_b.fit(x1,y1)

In [ ]:
dtc_s.fit(x2,y2)

In [ ]:
data['dtc']=[dtc_b.score(x3,y3),dtc_s.score(x4,y4)]

In [ ]:
dtc_b.score(x3,y3)

In [ ]:
data

In [ ]:
type(data.keys())

## Visualizing the accuracy comparison between different models

In [ ]:
import matplotlib.pyplot as plt
models = (data.keys())
performances = (data.values())

fig, ax = plt.subplots()
bar_width = 0.35
index = range(len(models))

bars1 = ax.bar(index, [perf[0] for perf in performances], bar_width, label='Buy')

bars2 = ax.bar([i + bar_width for i in index], [perf[1] for perf in performances], bar_width, label='Sell')

ax.set_xlabel('Models')
ax.set_ylabel('Performance')
ax.set_title('Accuracy of different models on buy and sell dataset')
ax.set_xticks([i + bar_width / 2 for i in index])
ax.set_xticklabels(models)
ax.set_ylim(0, 1)
ax.legend()

plt.show()

In [ ]:
list_buy = dtc_b.predict(x3)

In [ ]:
len(list_buy)

In [ ]:
list_sell = dtc_s.predict(x4)

In [ ]:
len(list_sell)

## Merging the Buy and Sell signals

In [ ]:
df1 = pd.read_csv('Buy_test_data_with_label.csv')

In [ ]:
df1.head()

In [ ]:
df_temp = pd.read_csv('Sell_Test_data_with_label.csv')

In [ ]:
len(df_temp)

In [ ]:
df_temp.head()

In [ ]:
df1 = pd.DataFrame()

In [ ]:
df2 = pd.DataFrame()

In [ ]:
df1['buy_signal'] = list_buy

In [ ]:
len(df1) , len(df_temp)

In [ ]:
df1['datetime'] = df_temp['datetime']

In [ ]:
df2['Sell_signal'] = list_sell

In [ ]:
'''long == -1 not inmarket , long = 1 , that is long trade is going on and long = 0 short trade is going on'''
def merge(df1,df2):
    list = [0]*len(df1)
    long = -1
    inmarket = 0
    for i in range(len(df1)):
        # print(df1['datetime'][p],df2['datetime'][p])
        if inmarket == 0:
            if df1['buy_signal'][i] == 1 and df2['Sell_signal'][i] == -1:
                continue
            elif df1['buy_signal'][i] == 0 and df2['Sell_signal'][i]==0:
                continue
            elif df1['buy_signal'][i] == 1:
                inmarket = 1
                long = 1
                list[i] = 1
            elif df2['Sell_signal'][i] == -1:
                inmarket = 1
                long = 0
                list[i] = -1
        elif inmarket == 1:
            if long == 1:
                if df1['buy_signal'][i] == 0 or df2['Sell_signal'][i] ==-1:
                    inmarket = 0
                    long = -1
                    list[i] = -1
            elif long == 0:
                if df1['buy_signal'][i] == 1 or df2['Sell_signal'][i] == 0:
                    inmarket = 0
                    long = -1
                    list[i] = 1
    return list

In [ ]:
# below cell contain merge function to merge buy and sell signal

In [ ]:
signals = merge(df1,df2)

In [ ]:
signals

In [ ]:
df1.head()

In [ ]:
df2.head()

In [ ]:
df = pd.DataFrame()

In [ ]:
df['datetime'] = df1['datetime']

In [ ]:
df['buy_signal'] = df1['buy_signal']

In [ ]:
df['sell_signal'] = df2['Sell_signal']

In [ ]:
df['open'] = df_temp['open']

In [ ]:
df['close'] = df_temp['close']

In [ ]:
df['high'] = df_temp['high']

In [ ]:
df['low'] = df_temp['low']

In [ ]:
df['volume'] = df_temp['volume']

In [ ]:
df['signals'] = signals

In [ ]:
df.head()

In [ ]:
df.to_csv('Long-Short-Signals.csv')

In [ ]:
df.head(50)

## Checking the backtested results
The actual results of the backtesting code would be far greater than the result shown, as the rate in which money increase is far greater than the result atleast 10X of the shown result
The actual calculation includes the involvement of concepts like option greeks , strike price and all , that may change the centre of the project

1. Number of Trades (done)
2. Number of long Trades (done)
3. Number of Short Trades (done)
4. Win Rate (done)
5. Number of profitable Trades (done)
6. Number of long profitable Trades (done)
7. Number of short profitable Trades (done)
8. Total returns   (done)
9. Returns from long trades (done)
10. Returns from short trades (done)

In [ ]:
df = pd.read_csv('Long-Short-Signals.csv')

In [ ]:
len(df)

In [ ]:
df.head()

In [ ]:
df.set_index('datetime',inplace=True)

In [ ]:
df.head()

In [ ]:
def SumColumn(df):
    df['sum'] = 0
    df['sum'][0] = df['signals'][0]
    for i in range(1,len(df)):
        df['sum'][i] = df['sum'][i-1] + df['signals'][i]
    return df

In [ ]:
df = SumColumn(df)

In [ ]:
df.head(50)

In [ ]:
def NumOfTrades(df):
    total = 0
    long = 0
    short = 0
    for i in range(len(df)):
        if df['signals'][i] == 1 and df['sum'][i] == 1:
            long += 1
        if df['signals'][i] == -1 and df['sum'][i] == -1:
            short += 1
    return long , short , long+ short

In [ ]:
long_trades , short_trades , total_trades = NumOfTrades(df)

In [ ]:
long_trades,short_trades,total_trades

In [ ]:
df_full = pd.read_csv('Buy_Train_data_with_label.csv')

In [ ]:
df_full.head(2)

In [ ]:
df_full.isna().sum()

In [ ]:
len(df) , len(df_full)

In [ ]:
# df['open'] = df_full['open']
# df['close'] = df_full['close']
# df['low'] = df_full['low']
# df['high'] = df_full['high']
# df['volume'] = df_full['volume']

In [ ]:
df.head()

In [ ]:
for i in range(len(df)):
    df['open'][i] = df_full['open'][i]
    df['close'][i] = df_full['close'][i]

In [ ]:
# initital is the amount 
def long_return(df , initial):
    profit = 0
    entry = -1
    long_profit_trades = 0
    max_profit = 0
    list = []
    df['long_returns'] = -1
    for i in range(len(df)):
        if df['sum'][i] == 1 and df['signals'][i] == 1:
            entry = df['open'][i]
        elif df['sum'][i]==0 and df['signals'][i] == -1:
            final = (df['close'][i]/entry)*initial
            df['long_returns'][i] = (final-initial)/initial
            profit += (final-initial)
            list.append(final-initial)
            max_profit = max(max_profit , final-initial)
            if( final > initial ):
                long_profit_trades += 1
    return profit , long_profit_trades , max_profit , list ,  df

In [ ]:
total_long_profit , Num_long_profit_trades , max_long_profit , list ,  df = long_return(df , 1000)

In [ ]:
total_long_profit , Num_long_profit_trades , max_long_profit

In [ ]:
list

In [ ]:
def short_return(df , initial):
    profit = 0
    entry = -1
    short_profit_trades = 0
    max_profit = 0
    list = []
    df['short_returns'] = -1
    for i in range(len(df)):
        if df['sum'][i] == -1 and df['signals'][i] == -1:
            entry = df['open'][i]
        elif df['sum'][i] == 0 and df['signals'][i] == 1:
            final = (df['close'][i]/entry)*initial
            profit += (initial-final)
            df['short_returns'][i] = profit/initial
            list.append(initial-final)
            max_profit = max(max_profit , initial-final)
            if initial > final :
                short_profit_trades += 1
    return profit , short_profit_trades , max_profit , list , df
            

In [ ]:
total_short_profit , Num_short_profit_trades , short_max_profit , list , df = short_return(df , 1000)

In [ ]:
total_short_profit , Num_short_profit_trades , short_max_profit 

In [ ]:
def win_rate(long_trades , short_trades , Num_long_profit_trades , Num_short_profit_trades ):
    long_win_rate = Num_long_profit_trades/long_trades
    short_win_rate = Num_short_profit_trades/short_trades
    win_rate = (Num_long_profit_trades+Num_short_profit_trades)/(long_trades+short_trades)
    return long_win_rate , short_win_rate , win_rate

In [ ]:
long_win_rate , short_win_rate , win_rate = win_rate(long_trades , short_trades , Num_long_profit_trades , Num_short_profit_trades )

In [ ]:
long_win_rate , short_win_rate , win_rate

In [ ]:
df.head()

In [ ]:
df.to_csv('Data_for_backtesting.csv')

## Using Backtesting module

In [ ]:
from backtesting import Backtest, Strategy

In [ ]:
df = pd.read_csv('Data_for_backtesting.csv')

In [ ]:
df['Close'] = df['close']
df['Open'] = df['open']
df['Low'] = df['low']
df['High'] = df['high']
df['Volume'] = df['volume']
df['Open'] /= 1e6
df['High'] /= 1e6
df['Low'] /= 1e6
df['Close'] /= 1e6

In [ ]:
df.head()

In [ ]:
class MyStrat(Strategy):
    def init(self):
        pass
    def next(self):
        if self.data['sum'] == 0:
            self.position.close()
        elif self.data['signals'] == -1 and self.data['sum'] == -1:
            self.sell()
        elif self.data['signals'] == 1 and self.data['sum'] == 1:
            self.buy()


In [ ]:
bt = Backtest(df, MyStrat, cash=1000, commission=0.001,trade_on_close=True)
result = bt.run()
bt.plot(superimpose=True)